Not used. Errors kept occuring due incompatible versions. We did not end up finding compatible versions and therefore this notebook was not used to acheive the final result

# Imports

In [ ]:
!pip install scikeras==0.11.0 scikit-learn==1.2.2

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
import scikeras
import tensorflow as tf

from sklearn import metrics

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from scikeras.wrappers import KerasClassifier
from sklearn.model_selection  import GridSearchCV

from google.colab import drive
drive.mount('/content/drive')

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

# Load dataset

In [ ]:
X_train = np.load("/content/drive/MyDrive/DAT255_Semesterproject/processed_data/X_train.npy")
X_test = np.load("/content/drive/MyDrive/DAT255_Semesterproject/processed_data/X_test.npy")
y_train = np.load("/content/drive/MyDrive/DAT255_Semesterproject/processed_data/y_train.npy")
y_test = np.load("/content/drive/MyDrive/DAT255_Semesterproject/processed_data/y_test.npy")


In [ ]:
n_classes = 6

LABELS = [
    "JUMPING",
    "JUMPING_JACKS",
    "BOXING",
    "WAVING_2HANDS",
    "WAVING_1HAND",
    "CLAPPING_HANDS"
]

# Convert labels to categorical
y_train_cat = to_categorical(y_train, num_classes=n_classes)


# Using randomsearch

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np

# Create synthetic classification dataset
X, y = make_classification(n_samples=500, n_features=10, n_informative=5, n_classes=3)
X = np.reshape(X, (X.shape[0], 5, 2))  # reshape for GRU: (samples, timesteps, features)
y = y.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

def build_model(input_shape=(5, 2), n_classes=3):
    model = Sequential([
        GRU(32, input_shape=input_shape),
        Dense(n_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

clf = KerasClassifier(
    model=build_model,
    model__input_shape=(5, 2),
    model__n_classes=3,
    epochs=3,
    batch_size=32,
    verbose=0
)

clf.fit(X_train, y_train)
print("Test score:", clf.score(X_test, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Test score: 0.55


In [ ]:
import scikeras
import sklearn
print("scikeras:", scikeras.__version__)
print("scikit-learn:", sklearn.__version__)


scikeras: 0.13.0
scikit-learn: 1.6.1


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

def build_model(units_1=128, units_2=64, dropout_rate=0.3, learning_rate=0.0001, input_shape=None, n_classes=None):
    model = Sequential([
        GRU(units_1, input_shape=input_shape, return_sequences=True),
        Dropout(dropout_rate),
        GRU(units_2),
        Dropout(dropout_rate),
        Dense(64, activation='relu'),
        Dense(n_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# ✅ This is correct for scikeras >= 0.11
model = KerasClassifier(
    model=build_model,
    input_shape=(X_train.shape[1], X_train.shape[2]),
    n_classes=n_classes,
    verbose=0
)

param_dist = {
    'model__units_1': [64, 128, 256],
    'model__units_2': [32, 64, 128],
    'model__dropout_rate': [0.2, 0.3, 0.4, 0.5],
    'model__learning_rate': [1e-4, 5e-4, 1e-3],
    'batch_size': [32, 64],
    'epochs': [20, 30, 50]
}

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=5,
    scoring='accuracy',
    cv=2,
    verbose=2,
    n_jobs=1
)

random_search_result = random_search.fit(X_train, y_train)


AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [ ]:
print("Best Accuracy: {:.2f}%".format(random_search_result.best_score_ * 100))
print("Best Hyperparameters:", random_search_result.best_params_)